In [1]:
import os
import json
from typing import List
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.nn.init import xavier_normal_, constant_
from torch.utils.data import DataLoader, Dataset
from catalyst import dl, metrics
from catalyst.utils import set_global_seed

In [2]:
class InteractionsDataset(Dataset):
    def __init__(self, interactions_pickle_path: str):
        data = pd.read_pickle(interactions_pickle_path)
        users = data['user'].to_numpy()
        tracks = data['track'].to_numpy()

        i = torch.from_numpy(np.stack((users, tracks)).astype("int64"))
        v = torch.ones(data.shape[0])

        self.interactions = torch.sparse.FloatTensor(i, v)

    def __len__(self):
        return len(self.interactions)

    def __getitem__(self, idx):
        return self.interactions[idx].to_dense()

In [3]:
def collate_fn(batch: List[torch.Tensor]) -> torch.Tensor:
    return {"inputs": torch.stack(batch), "targets": torch.stack(batch)}

In [4]:
class MultiDAE(nn.Module):
    def __init__(self, p_dims, q_dims=None, dropout=0.5):
        super().__init__()
        self.p_dims = p_dims
        if q_dims:
            assert q_dims[0] == p_dims[-1], "In and Out dimensions must equal to each other"
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q- network mismatches."
            self.q_dims = q_dims
        else:
            self.q_dims = p_dims[::-1]

        self.dims = self.q_dims + self.p_dims[1:]
        self.layers = nn.ModuleList([nn.Linear(d_in, d_out) for
            d_in, d_out in zip(self.dims[:-1], self.dims[1:])])
        self.drop = nn.Dropout(dropout)
        
        self.init_weights()
    
    def forward(self, input):
        h = F.normalize(input)
        h = self.drop(h)

        for i, layer in enumerate(self.layers):
            h = layer(h)
            if i != len(self.layers) - 1:
                h = torch.tanh(h)
        return h

    def init_weights(self):
        for layer in self.layers:
            xavier_normal_(layer.weight.data)
            constant_(layer.bias.data, 0)

In [5]:
set_global_seed(42)

In [6]:
# For top_k tracks recommendation
top_k = 50

In [7]:
train_dataset = InteractionsDataset("train_data.pkl")
loaders = {
    "train": DataLoader(train_dataset, batch_size=64, collate_fn=collate_fn),
}

In [8]:
item_num = len(train_dataset[0])
model = MultiDAE([50, 300, item_num], dropout=0.5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
engine = dl.DeviceEngine()

In [9]:
callbacks = [
    dl.NDCGCallback("logits", "targets", [top_k]),
    dl.MAPCallback("logits", "targets", [top_k]),
    dl.MRRCallback("logits", "targets", [top_k]),
    dl.HitrateCallback("logits", "targets", [top_k]),
    dl.OptimizerCallback("loss", accumulation_steps=1),
]

In [10]:
runner = dl.SupervisedRunner(
    input_key="inputs", output_key="logits", target_key="targets", loss_key="loss"
)

In [11]:
runner.train(
  model=model,
  optimizer=optimizer,
  criterion=criterion,
  engine=engine,
  loaders=loaders, 
  num_epochs=30,
  verbose=True,
  timeit=False,
  callbacks=callbacks,
  logdir="./logs",
)

1/30 * Epoch (train):   0%|          | 0/157 [00:00<?, ?it/s]

train (1/30) hitrate50: 0.027579400061257187 | hitrate50/std: 0.0077996005370578134 | loss: 1140.9674562500006 | loss/mean: 1140.9674562500006 | loss/std: 125.08365167589554 | lr: 0.001 | map50: 0.08966458641737701 | map50/std: 0.022428504986206237 | momentum: 0.9 | mrr50: 0.11615213832110169 | mrr50/std: 0.03335677848999456 | ndcg50: 0.043895978221297254 | ndcg50/std: 0.01140951853831152
* Epoch (1/30) 


2/30 * Epoch (train):   0%|          | 0/157 [00:00<?, ?it/s]

train (2/30) hitrate50: 0.04689595823287962 | hitrate50/std: 0.015617750745945156 | loss: 1125.787861914063 | loss/mean: 1125.787861914063 | loss/std: 124.57489858696772 | lr: 0.001 | map50: 0.17120562121868133 | map50/std: 0.06382950139676609 | momentum: 0.9 | mrr50: 0.21209121668338773 | mrr50/std: 0.0820304246028037 | ndcg50: 0.08148553736209867 | ndcg50/std: 0.02866333384268424
* Epoch (2/30) 


3/30 * Epoch (train):   0%|          | 0/157 [00:00<?, ?it/s]

...

train (28/30) hitrate50: 0.3918063077926633 | hitrate50/std: 0.02384109420571127 | loss: 819.5523734375001 | loss/mean: 819.5523734375001 | loss/std: 95.59280164467988 | lr: 0.001 | map50: 0.8424814487457277 | map50/std: 0.014963895949810067 | momentum: 0.9 | mrr50: 0.9852712375640865 | mrr50/std: 0.011753972723081381 | ndcg50: 0.6914725805282593 | ndcg50/std: 0.021015259986566774
* Epoch (28/30) 


29/30 * Epoch (train):   0%|          | 0/157 [00:00<?, ?it/s]

train (29/30) hitrate50: 0.3985554338455199 | hitrate50/std: 0.02365801707628632 | loss: 812.0135927734373 | loss/mean: 812.0135927734373 | loss/std: 94.4849641290902 | lr: 0.001 | map50: 0.8469265758514404 | map50/std: 0.013835580788707568 | momentum: 0.9 | mrr50: 0.9853995681762691 | mrr50/std: 0.012053625599193162 | ndcg50: 0.7015929243087765 | ndcg50/std: 0.02172707027723074
* Epoch (29/30) 


30/30 * Epoch (train):   0%|          | 0/157 [00:00<?, ?it/s]

train (30/30) hitrate50: 0.4046434797286987 | hitrate50/std: 0.02378385468192556 | loss: 804.8221525390619 | loss/mean: 804.8221525390619 | loss/std: 93.30984652992261 | lr: 0.001 | map50: 0.852378803730011 | map50/std: 0.013328375184972154 | momentum: 0.9 | mrr50: 0.986595006942749 | mrr50/std: 0.010674718362603947 | ndcg50: 0.7116232017517093 | ndcg50/std: 0.02157568697793173
* Epoch (30/30) 
Top best models:
logs\checkpoints/train.30.pth	30.0000


In [12]:
%%time
with open("recommendations_30.json", "w") as rf:    
    for batch, prediction in enumerate(runner.predict_loader(loader=loaders["train"])):
        preds = prediction["logits"].detach().cpu().numpy()
        for i, pred in enumerate(preds):
            user = loaders["train"].batch_size * batch + i
            pred_tracks = np.argsort(pred)[::-1][:top_k]
            
            recommendation = {
                "user": user,
                "tracks": pred_tracks.tolist(),
            }
            rf.write(json.dumps(recommendation) + "\n")

CPU times: total: 5min 45s
Wall time: 47.5 s
